In [39]:
import mindspore as ms
from mindspore import Tensor
import numpy as np
import mindspore.ops.operations as P
import mindspore.nn as nn
from mindspore.ops import operations as P
from mindspore.ops import functional as F
from mindspore.ops import composite as C

In [21]:
labels = Tensor(np.random.rand(16,2100), ms.float32)
bbox_preds = Tensor(np.random.rand(16,2100,32), ms.float32)
less = P.Less()

In [37]:
mask = less(0.5,labels)
mask

Tensor(shape=[16, 2100], dtype=Bool, value=
[[ True,  True, False ... False,  True,  True],
 [False, False,  True ... False, False, False],
 [ True,  True, False ...  True, False, False],
 ...
 [False,  True,  True ...  True, False, False],
 [ True,  True, False ...  True,  True,  True],
 [ True,  True,  True ... False, False,  True]])

In [30]:
class GIouLossII(nn.Cell):
    def __init__(self, eps=1e-6, reduction='mena', loss_weight=2.0):
        super(GIouLossII, self).__init__()
        self.reduction = reduction
        self.loss_weight = loss_weight
        self.maximum = P.Maximum()
        self.minimum = P.Minimum()
        self.eps = Tensor(eps, ms.float32)
        self.value_zero = Tensor(0, ms.float32)

    def construct(self, boxes1, boxes2):
        boxes1Area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
        boxes2Area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])
        left_up = self.maximum(boxes1[..., :2], boxes2[..., :2])
        right_down = self.minimum(boxes1[..., 2:], boxes2[..., 2:])
        inter_section = self.maximum(right_down - left_up, self.value_zero)
        inter_area = inter_section[..., 0] * inter_section[..., 1]
        union_area = boxes1Area + boxes2Area - inter_area
        ious = self.maximum(1.0 * inter_area / union_area, self.eps)
        enclose_left_up = self.minimum(boxes1[..., :2], boxes2[..., :2])
        enclose_right_down = self.maximum(boxes1[..., 2:], boxes2[..., 2:])
        enclose = self.maximum(enclose_right_down - enclose_left_up, self.value_zero)
        enclose_area = enclose[..., 0] * enclose[..., 1]
        gious_loss = ious - 1.0 * (enclose_area - union_area) / enclose_area
        gious_loss = 1 - gious_loss
        gious_loss = self.loss_weight * gious_loss
        return gious_loss

class Giou(nn.Cell):
    """Calculating giou"""
    def __init__(self):
        super(Giou, self).__init__()
        self.cast = P.Cast()
        self.reshape = P.Reshape()
        self.min = P.Minimum()
        self.max = P.Maximum()
        self.concat = P.Concat(axis=1)
        self.mean = P.ReduceMean()
        self.div = P.RealDiv()
        self.eps = 0.000001

    def construct(self, box_p, box_gt):
        """construct method"""
        box_p_area = (box_p[..., 2:3] - box_p[..., 0:1]) * (box_p[..., 3:4] - box_p[..., 1:2])
        box_gt_area = (box_gt[..., 2:3] - box_gt[..., 0:1]) * (box_gt[..., 3:4] - box_gt[..., 1:2])
        x_1 = self.max(box_p[..., 0:1], box_gt[..., 0:1])
        x_2 = self.min(box_p[..., 2:3], box_gt[..., 2:3])
        y_1 = self.max(box_p[..., 1:2], box_gt[..., 1:2])
        y_2 = self.min(box_p[..., 3:4], box_gt[..., 3:4])
        intersection = (y_2 - y_1) * (x_2 - x_1)
        xc_1 = self.min(box_p[..., 0:1], box_gt[..., 0:1])
        xc_2 = self.max(box_p[..., 2:3], box_gt[..., 2:3])
        yc_1 = self.min(box_p[..., 1:2], box_gt[..., 1:2])
        yc_2 = self.max(box_p[..., 3:4], box_gt[..., 3:4])
        c_area = (xc_2 - xc_1) * (yc_2 - yc_1)
        union = box_p_area + box_gt_area - intersection
        union = union + self.eps
        c_area = c_area + self.eps
        iou = self.div(self.cast(intersection, ms.float32), self.cast(union, ms.float32))
        res_mid0 = c_area - union
        res_mid1 = self.div(self.cast(res_mid0, ms.float32), self.cast(c_area, ms.float32))
        giou = iou - res_mid1
        giou = 1 - giou
        # giou = C.clip_by_value(giou, -1.0, 1.0)
        return giou

In [38]:
boxes1 = Tensor(np.random.rand(16,2100,4), ms.float32)
boxes2 = Tensor(np.random.rand(16,2100,4), ms.float32)
giou = GIouLossII()
yolo_giou = Giou()
yolo_giou(boxes1,boxes2)
yolo_giou(boxes1,boxes2).squeeze() * mask

Tensor(shape=[16, 2100], dtype=Float32, value=
[[3.65924382e+000, -9.53432941e+000, -0.00000000e+000 ... 0.00000000e+000, 9.19221461e-001, 8.82639885e+000],
 [-0.00000000e+000, 0.00000000e+000, 3.36038160e+000 ... -0.00000000e+000, -0.00000000e+000, 0.00000000e+000],
 [3.42953753e+000, 1.20811439e+000, 0.00000000e+000 ... 4.08004665e+000, 0.00000000e+000, 0.00000000e+000],
 ...
 [-0.00000000e+000, -8.08080435e-002, 5.06755590e+000 ... -1.05221844e+001, 0.00000000e+000, 0.00000000e+000],
 [7.38240147e+000, 3.61924696e+000, -0.00000000e+000 ... -8.82824659e-002, -1.51215317e+002, 1.19135571e+000],
 [-9.86533833e+000, 1.11915207e+000, -6.76851463e+000 ... 0.00000000e+000, -0.00000000e+000, 9.82322991e-001]])

In [35]:
yolo_giou(boxes1,boxes2)

Tensor(shape=[16, 2100, 1], dtype=Float32, value=
[[[2.93751550e+000],
  [3.52184105e+000],
  [1.92462788e+001],
  ...
  [-7.85906911e-001],
  [2.59374952e+000],
  [1.24731374e+000]],
 [[-1.74807644e+000],
  [-6.83522320e+000],
  [-1.02864227e+001],
  ...
  [-1.02001882e+000],
  [1.02090168e+000],
  [-5.15212917e+000]],
 [[1.97709262e+000],
  [3.27673292e+000],
  [-2.48940349e-001],
  ...
  [-1.96171165e+000],
  [7.14198875e+000],
  [-1.01483954e+003]],
 ...
 [[1.76815808e-001],
  [-1.25984812e+001],
  [3.48497510e+000],
  ...
  [1.01597691e+000],
  [-1.59598303e+000],
  [3.17451334e+000]],
 [[-2.98324188e+002],
  [6.97393179e-001],
  [3.55034232e+000],
  ...
  [3.94666195e-003],
  [-1.06493080e+002],
  [1.89908638e+001]],
 [[1.02244937e+000],
  [4.35289621e+000],
  [2.55364990e+000],
  ...
  [-1.65686727e-001],
  [2.04250193e+000],
  [8.38040888e-001]]])

In [105]:
class QualityFocalLoss(nn.Cell):
    def __init__(self, beta=2.0):
        super(QualityFocalLoss, self).__init__()
        self.sigmoid = P.Sigmoid()
        self.sigmiod_cross_entropy = P.SigmoidCrossEntropyWithLogits()
        self.pow = P.Pow()
        self.abs = P.Abs()
        self.onehot = P.OneHot()
        self.on_value = Tensor(1.0, ms.float32)
        self.off_value = Tensor(0.0, ms.float32)
        self.tile = P.Tile()
        self.expand_dims = P.ExpandDims()
        self.beta = beta

    def construct(self, logits, label, score):
        logits_sigmoid = self.sigmoid(logits)
        label = self.onehot(label, F.shape(logits)[-1], self.on_value, self.off_value)
        score = self.tile(self.expand_dims(score, -1), (1, 1, F.shape(logits)[-1]))
        label = label * score
        sigmiod_cross_entropy = self.sigmiod_cross_entropy(logits, label)
        modulating_factor = self.pow(self.abs(label - logits_sigmoid), self.beta)
        qfl_loss = sigmiod_cross_entropy * modulating_factor
        return qfl_loss

In [106]:
qfl = QualityFocalLoss()
np.random.seed(1)

In [107]:
logits = Tensor(np.random.rand(1,4,5), ms.float32)

In [115]:
label = Tensor([[4,-1,-2,0,2,79]], ms.int64)
onehot = P.OneHot()
onehot(label, 80, Tensor(1.0, ms.float32), Tensor(0.0, ms.float32))

Tensor(shape=[1, 6, 80], dtype=Float32, value=
[[[0.00000000e+000, 0.00000000e+000, 0.00000000e+000 ... 0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
  [0.00000000e+000, 0.00000000e+000, 0.00000000e+000 ... 0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
  [0.00000000e+000, 0.00000000e+000, 0.00000000e+000 ... 0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
  [1.00000000e+000, 0.00000000e+000, 0.00000000e+000 ... 0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
  [0.00000000e+000, 0.00000000e+000, 1.00000000e+000 ... 0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
  [0.00000000e+000, 0.00000000e+000, 0.00000000e+000 ... 0.00000000e+000, 0.00000000e+000, 1.00000000e+000]]])

In [109]:
score = Tensor(np.random.rand(1,4), ms.float32)

In [110]:
label.shape

(1, 4)

In [111]:
qfl(logits, label, score)

(Tensor(shape=[1, 4, 5], dtype=Float32, value=
 [[[3.35443079e-001, 5.05353451e-001, 1.73320919e-001, 2.82926857e-001, 4.54626903e-002],
   [1.29021138e-001, 2.36064985e-001, 3.01982492e-001, 3.25709045e-001, 3.98207128e-001],
   [3.36498946e-001, 4.83358204e-001, 4.15477604e-002, 6.11653805e-001, 1.81606799e-001],
   [4.74296689e-001, 3.35580468e-001, 4.09142345e-001, 1.65400915e-002, 2.40564123e-001]]]),
 Tensor(shape=[1, 4, 5], dtype=Float32, value=
 [[[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 8.00744593e-001],
   [9.68261600e-001, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
   [0.00000000e+000, 0.00000000e+000, 3.13424170e-001, 0.00000000e+000, 0.00000000e+000],
   [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 6.92322612e-001, 0.00000000e+000]]]))

In [141]:
class DistributionFocalLossII(nn.Cell):
    def __init__(self, loss_weight=0.25):
        super(DistributionFocalLossII, self).__init__()
        self.loss_weight = loss_weight
        self.cross_entropy = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='none')
        self.cast = P.Cast()
        self.reduce_sum = P.ReduceSum()

    def construct(self, pred, label):
        dis_left = self.cast(label, ms.int32)
        dis_right = dis_left + 1
        weight_left = self.cast(dis_right, ms.float32) - label
        weight_right = label - self.cast(dis_left, ms.float32)
        dfl_loss = (
                self.cross_entropy(pred, dis_left) * weight_left
                + self.cross_entropy(pred, dis_right) * weight_right)
        # dfl_loss = dfl_loss * self.loss_weight
        return dfl_loss

In [78]:
dfl = DistributionFocalLossII()

In [142]:
np.random.seed(1)
label = Tensor([0,0,0,0], ms.int64)
logits = Tensor(np.zeros((4,8)), ms.float32)

In [143]:
dfl(logits, label)

Tensor(shape=[4], dtype=Float32, value= [2.07944155e+000, 2.07944155e+000, 2.07944155e+000, 2.07944155e+000])

In [136]:
ind = Tensor([True, False, False, True])
label[ind]

IndexError: The tensor index must be int type, but got (mindspore.bool_,).

In [126]:

class Iou(nn.Cell):
    def __init__(self):
        super(Iou, self).__init__()
        self.cast = P.Cast()
        self.min = P.Minimum()
        self.max = P.Maximum()
        self.div = P.RealDiv()
        self.eps = 0.000001

    def construct(self, box_p, box_gt):
        """construct method"""
        box_p_area = (box_p[..., 2:3] - box_p[..., 0:1]) * (box_p[..., 3:4] - box_p[..., 1:2])
        box_gt_area = (box_gt[..., 2:3] - box_gt[..., 0:1]) * (box_gt[..., 3:4] - box_gt[..., 1:2])
        x_1 = self.max(box_p[..., 0:1], box_gt[..., 0:1])
        x_2 = self.min(box_p[..., 2:3], box_gt[..., 2:3])
        y_1 = self.max(box_p[..., 1:2], box_gt[..., 1:2])
        y_2 = self.min(box_p[..., 3:4], box_gt[..., 3:4])
        intersection = (y_2 - y_1) * (x_2 - x_1)
        xc_1 = self.min(box_p[..., 0:1], box_gt[..., 0:1])
        xc_2 = self.max(box_p[..., 2:3], box_gt[..., 2:3])
        yc_1 = self.min(box_p[..., 1:2], box_gt[..., 1:2])
        yc_2 = self.max(box_p[..., 3:4], box_gt[..., 3:4])
        union = box_p_area + box_gt_area - intersection
        union = union + self.eps
        iou = self.div(self.cast(intersection, ms.float32), self.cast(union, ms.float32)).squeeze(-1)
        return iou

In [127]:
iou = Iou()

In [128]:
iou(boxes1,boxes2)

Tensor(shape=[16, 2100], dtype=Float32, value=
[[-9.91776586e-001, -7.69083023e-001, -1.64417827e+000 ... -8.33307385e-001, 8.07785541e-002, -4.82948631e-001],
 [-6.76408112e-001, -2.54682392e-001, -2.50901639e-001 ... -7.33016777e+000, -2.20871949e+000, -8.82815063e-001],
 [-3.23499262e-001, -3.02022398e-001, -8.54857683e-001 ... -5.39389253e-001, -7.29965746e-001, -9.65086579e-001],
 ...
 [9.23408926e-001, 1.08080816e+000, -2.94611263e+000 ... -1.85042799e+000, -2.92762220e-001, 5.04058935e-002],
 [-2.63358325e-001, -2.10003972e+000, 6.91856563e-001 ... 8.74416530e-001, 1.52215317e+002, -1.91355720e-001],
 [1.07933104e-001, -1.19152114e-001, -1.76416874e-001 ... -2.24564597e-001, 3.88602400e+000, 1.76770352e-002]])

In [139]:
import mindspore.numpy as mnp

In [140]:
mnp.where(ind,label)

TypeError: Expect input to be array like.

In [157]:
test = Tensor([[0,0,0,0,0,0,0,0,0,0], [11,12,13,14,15,16,17,18,19,20]])
ind = P.Tile()(Tensor([[False], [True]]),(1,5))
test.reshape((-1, 2))

Tensor(shape=[10, 2], dtype=Int32, value=
[[ 0,  0],
 [ 0,  0],
 [ 0,  0],
 ...
 [15, 16],
 [17, 18],
 [19, 20]])

In [160]:
ind.reshape(-1)

Tensor(shape=[10], dtype=Bool, value= [False, False, False, False, False,  True,  True,  True,  True,  True])

In [ ]:

def bbox_overlaps(bbox):
    ymin = np.maximum(y1, bbox[0])
    xmin = np.maximum(x1, bbox[1])
    ymax = np.minimum(y2, bbox[2])
    xmax = np.minimum(x2, bbox[3])
    # 并行化运算
    w = np.maximum(xmax - xmin, 0.)
    h = np.maximum(ymax - ymin, 0.)

    inter_vol = h * w
    union_vol = vol_anchors + (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]) - inter_vol
    iou = inter_vol / union_vol
    return np.squeeze(iou)

In [13]:
from mindspore import Tensor
a = Tensor([1,1,1,1,1])
b = Tensor([1,1,1,0,1])
a == b

Tensor(shape=[5], dtype=Bool, value= [ True,  True,  True, False,  True])

In [14]:
a.item()

ValueError: Can only convert an array of size 1 to a Python scalar

In [7]:
import mindspore.ops as ops

ops.equal(a,b)

Tensor(shape=[5], dtype=Bool, value= [ True,  True,  True, False,  True])

Tensor(shape=[5], dtype=Int32, value= [1, 1, 1, 1, 1])

In [16]:
b = Tensor([1])

In [18]:
b.value

AttributeError: 'Tensor' object has no attribute 'value'

In [21]:
import mindspore

x = mindspore.Tensor([0])
if x.value()==1:
    print("1")

AttributeError: 'Tensor' object has no attribute 'value'